In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from concurrent.futures import ThreadPoolExecutor
from time import sleep
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import json


Get all products at Category in Tiki.vn

In [2]:

# Đường dẫn lưu bookmark
BOOKMARK_PATH = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/bookmark.csv"

# Thư mục lưu file kết quả
OUTPUT_DIR = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

def load_bookmark(file_path):
    """Load danh sách liên kết đã cào từ file."""
    if os.path.exists(file_path):
        return set(pd.read_csv(file_path)['Links'])
    return set()

def save_bookmark(file_path, links):
    """Lưu danh sách liên kết đã cào vào file (thêm mới, không ghi đè)."""
    if os.path.exists(file_path):
        existing_links = load_bookmark(file_path)
        links = links.union(existing_links)  # Gộp liên kết cũ và mới
    df = pd.DataFrame({'Links': list(links)})
    df.to_csv(file_path, index=False)

def save_all_store_links(store_links):
    """Lưu tất cả các đường link cửa hàng vào một file Excel chung."""
    output_file = os.path.join(OUTPUT_DIR, "store_links.xlsx")
    df = pd.DataFrame({'Links': list(store_links)})
    df.to_excel(output_file, index=False)
    print(f"Đã lưu tất cả các liên kết cửa hàng vào file: {output_file}")

def scroll_page(driver, scroll_count=10, pause=1.5, offset=25):
    """Cuộn trang để tải thêm nội dung."""
    total_height = driver.execute_script("return document.body.scrollHeight")
    scroll_step = (total_height - offset) / scroll_count
    for i in range(1, scroll_count + 1):
        driver.execute_script(f"window.scrollTo(0, {i * scroll_step});")
        time.sleep(pause)
    time.sleep(2)  # Đợi 2 giây sau khi cuộn xong

def collect_product_links(driver, product_links):
    """Thu thập liên kết sản phẩm từ trang hiện tại."""
    product_elements = driver.find_elements(By.CSS_SELECTOR, ".styles__ProductItemContainerStyled-sc-bszvl7-0 a.style__ProductLink-sc-139nb47-2")
    for elem in product_elements:
        link = elem.get_attribute("href")
        if link and link not in product_links:
            product_links.add(link)
            print(f"Collected {len(product_links)} links")

def crawl_category_links(driver, category_links):
    """
    Duyệt qua các danh mục và thu thập các liên kết cửa hàng.
    Thu thập liên tục cho đến khi không còn nút "Xem thêm" hoặc không có sản phẩm mới.
    """
    all_store_links = set()

    for idx, href in enumerate(category_links, start=1):
        print(f"\nĐang truy cập danh mục {idx}: {href}")
        driver.get(href)
        time.sleep(random.uniform(2, 5))

        category_product_links = set()

        while True:
            # Lưu số lượng hiện tại để so sánh sau khi thu thập
            current_count = len(category_product_links)
            collect_product_links(driver, category_product_links)
            
            # Nếu không có liên kết mới được thêm, có thể đã tải hết sản phẩm hiện tại
            if len(category_product_links) == current_count:
                try:
                    # Thử tìm và click nút "Xem thêm" nếu có
                    button = driver.find_element(By.CSS_SELECTOR, '[data-view-id="category_infinity_view.more"]')
                    ActionChains(driver).move_to_element(button).click(button).perform()
                    time.sleep(random.uniform(2, 3))
                    scroll_page(driver, scroll_count=5, pause=random.uniform(1, 1.5))
                except NoSuchElementException:
                    print("Nút 'Xem thêm' không còn hoặc đã tải hết sản phẩm.")
                    break
            else:
                # Nếu có sản phẩm mới được thêm, thử cuộn trang để kích hoạt load thêm
                scroll_page(driver, scroll_count=5, pause=random.uniform(1, 1.5))
        
        print(f"Danh mục {idx} hoàn thành. Thu thập được {len(category_product_links)} liên kết.")
        # Cập nhật tất cả liên kết cửa hàng với liên kết từ danh mục hiện tại
        all_store_links = all_store_links.union(category_product_links)

    return all_store_links

def main():
    # Tải bookmark đã cào (nếu có)
    crawled_links = load_bookmark(BOOKMARK_PATH)

    # Khởi tạo WebDriver
    driver = webdriver.Chrome()
    driver.get("https://tiki.vn")
    driver.implicitly_wait(5)

    # Tìm tất cả các thẻ <a> bên trong class "styles__StyledListItem-sc-w7gnxl-0"
    category_elements = driver.find_elements(By.CSS_SELECTOR, ".styles__StyledListItem-sc-w7gnxl-0 a")
    category_links = [link.get_attribute("href") for link in category_elements if link.get_attribute("href") not in crawled_links]
    category_links = category_links[1:26]  # Lấy danh mục từ 2 đến 26

    # Thu thập liên kết sản phẩm từ các danh mục và gom chúng lại
    all_store_links = crawl_category_links(driver, category_links)

    print("\nHoàn thành thu thập liên kết sản phẩm.")
    print(f"Tổng số liên kết thu thập được: {len(all_store_links)}")

    # Lưu tất cả các liên kết cửa hàng vào 1 file Excel chung
    save_all_store_links(all_store_links)

    # Nếu cần cập nhật bookmark thì lưu lại các liên kết danh mục đã cào (tuỳ chọn)
    save_bookmark(BOOKMARK_PATH, set(category_links))

    # Đóng trình duyệt
    driver.quit()

if __name__ == "__main__":
    main()



Đang truy cập danh mục 1: https://tiki.vn/nha-cua-doi-song/c1883
Collected 1 links
Collected 2 links
Collected 3 links
Collected 4 links
Collected 5 links
Collected 6 links
Collected 7 links
Collected 8 links
Collected 9 links
Collected 10 links
Collected 11 links
Collected 12 links
Collected 13 links
Collected 14 links
Collected 15 links
Collected 16 links
Collected 17 links
Collected 18 links
Collected 19 links
Collected 20 links
Collected 21 links
Collected 22 links
Collected 23 links
Collected 24 links
Collected 25 links
Collected 26 links
Collected 27 links
Collected 28 links
Collected 29 links
Collected 30 links
Collected 31 links
Collected 32 links
Collected 33 links
Collected 34 links
Collected 35 links
Collected 36 links
Collected 37 links
Collected 38 links
Collected 39 links
Collected 40 links
Collected 41 links
Collected 42 links
Collected 43 links
Collected 44 links
Collected 45 links
Collected 46 links
Collected 47 links
Collected 48 links
Collected 49 links
Collected 50

Access to Get Store links

In [ ]:
import pandas as pd
import threading
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import load_workbook

# Đường dẫn file Excel
input_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_links.xlsx"
output_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_seller_links_1.xlsx"
error_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/Still not collected seller yet_1.xlsx"

# Đọc file Excel và lấy 20 dòng đầu tiên
df = pd.read_excel(input_file)
links = df["Links"].tolist()[13700:14000]
# 1800:2500
# Sử dụng tập hợp để tránh trùng lặp seller links
seller_links_set = set()

# Danh sách lưu các URL bị lỗi
error_links = []

# Khóa để đảm bảo ghi file an toàn khi dùng đa luồng
lock = threading.Lock()

# Hàm xử lý từng link
def process_link(url):
    global seller_links_set, error_links

    # Khởi tạo Selenium WebDriver trong từng luồng
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.implicitly_wait(10)

    try:
        print(f"🔹 Đang truy cập: {url}")
        driver.get(url)

        # **Thực hiện reload trang sau khi truy cập**
        driver.refresh()

        # Chờ trang load hoàn toàn sau khi reload (timeout 10s)
        WebDriverWait(driver, 10).until(
            # EC.presence_of_element_located((By.CSS_SELECTOR, "span.seller-name div.sc-d1e96cea-0.jBNFDc a"))
            EC.presence_of_element_located((By.CSS_SELECTOR, "span.seller-name div.SellerName__SellerNameStyled-sc-5d1cxl-0.myqYs a"))

        )

        # Tìm seller link
        # seller_element = driver.find_element(By.CSS_SELECTOR, "span.seller-name div.sc-d1e96cea-0.jBNFDc a")
        seller_element = driver.find_element(By.CSS_SELECTOR, "span.seller-name div.SellerName__SellerNameStyled-sc-5d1cxl-0.myqYs a")

        seller_href = seller_element.get_attribute("href")
        print(f"  ✅ Seller link: {seller_href}")

        # Dùng khóa để đảm bảo an toàn khi ghi file
        with lock:
            if seller_href not in seller_links_set:
                seller_links_set.add(seller_href)

                # Đọc file hiện tại (nếu có)
                try:
                    existing_df = pd.read_excel(output_file)
                except FileNotFoundError:
                    existing_df = pd.DataFrame(columns=["Seller Links"])

                # Thêm dữ liệu mới
                new_df = pd.DataFrame({"Seller Links": [seller_href]})
                final_df = pd.concat([existing_df, new_df], ignore_index=True)

                # Ghi vào file Excel
                with pd.ExcelWriter(output_file, mode="w", engine="openpyxl") as writer:
                    final_df.to_excel(writer, index=False)

    except TimeoutException:
        print(f"  ⚠️ Trang {url} load quá lâu, bỏ qua.")
        save_error(url, "Load quá lâu")
    except NoSuchElementException:
        print(f"  ❌ Không tìm thấy seller link trên trang: {url}")
        save_error(url, "Không tìm thấy seller link")
    except Exception as e:
        print(f"  ❌ Lỗi khi xử lý {url}: {e}")
        save_error(url, str(e))
    finally:
        driver.quit()

# Hàm lưu các URL bị lỗi vào file "Still not collected seller yet.xlsx"
def save_error(url, error_message):
    with lock:
        try:
            existing_error_df = pd.read_excel(error_file)
        except FileNotFoundError:
            existing_error_df = pd.DataFrame(columns=["URL", "Lỗi"])

        new_error_df = pd.DataFrame({"URL": [url], "Lỗi": [error_message]})
        final_error_df = pd.concat([existing_error_df, new_error_df], ignore_index=True)

        with pd.ExcelWriter(error_file, mode="w", engine="openpyxl") as writer:
            final_error_df.to_excel(writer, index=False)

# Sử dụng ThreadPoolExecutor để chạy đa luồng
max_threads = 5  # Số luồng tối đa có thể điều chỉnh
with ThreadPoolExecutor(max_workers=max_threads) as executor:
    executor.map(process_link, links)

print(f"\n✅ Đã hoàn thành. Dữ liệu lỗi được lưu vào: {error_file}")


🔹 Đang truy cập: https://tiki.vn/giay-loc-ca-phe-hario-100-cai-v60-vcf-02-100w-p117059826.html?spid=117059827
🔹 Đang truy cập: https://tiki.vn/dien-thoai-vivo-y18-4gb-128gb-hang-chinh-hang-p275775887.html?spid=275775888
🔹 Đang truy cập: https://tiki.vn/tui-chong-soc-laptop-macbook-agva-jersey-13inch-kich-thuoc-35-x-2-5-x-26-cm-ma-san-pham-slv338-3-mau-xam-xanh-den-p153978796.html?spid=153978810
🔹 Đang truy cập: https://tiki.vn/ruou-blended-whisky-johnnie-walker-double-black-40-750ml-co-hop-p186267263.html?spid=186267265
🔹 Đang truy cập: https://tiki.vn/thiet-bi-phat-song-wifi-tenda-n300-hang-chinh-hang-p950026.html?spid=272455472
  ✅ Seller link: https://tiki.vn/cua-hang/the-gioi-di-dong-official?source_screen=product_detail&source_engine=organic
  ✅ Seller link: https://tiki.vn/cua-hang/hario-official-store?source_screen=product_detail&source_engine=organic
  ✅ Seller link: https://tiki.vn/cua-hang/vua-balo?source_screen=product_detail&source_engine=organic
  ✅ Seller link: https://ti

Check Duplicate

In [16]:
import pandas as pd
df_new = pd.read_excel(r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/Still not collected seller yet.xlsx")
df_new.drop_duplicates(subset ="URL", keep = 'first', inplace = True)
df_new.to_excel(r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/Still not collected seller yet.xlsx", index=False)
df_new

,URL
0,https://tiki.vn/tham-taplo-da-cacbon-mitsubish...
1,https://tiki.vn/chuot-khong-day-dareu-lm115g-s...
2,https://tiki.vn/combo-nhot-liqui-moly-10w-40-s...
3,https://tiki.vn/hop-ty-tay-hyundai-i10-co-usb-...
4,https://tiki.vn/vi-dung-the-card-holder-nam-nu...
...,...
599,https://tiki.vn/nam-hong-chi-linh-chi-rung-vie...
600,https://tiki.vn/hop-dung-khan-giay-kiem-bang-s...
601,https://tiki.vn/tui-chong-soc-bao-da-laptop-da...
602,https://tiki.vn/tui-dung-laptop-14-inch-deo-va...


In [4]:
import pandas as pd
df_new = pd.read_excel(r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_seller_links.xlsx")
df_new.drop_duplicates(subset ="Seller Links", keep = 'first', inplace = True)
# df_new.to_excel(r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_seller_links.xlsx")

Get Infomation each Product in store

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, WebDriverException, ElementClickInterceptedException
from bs4 import BeautifulSoup  # Import BeautifulSoup để phân tích HTML

# Hàm cuộn trang có giới hạn (chừa lại 1/7 trang)
def scroll_page(driver, scroll_count=5, pause=1.5, offset=25):
    """Cuộn trang để tải thêm nội dung."""
    total_height = driver.execute_script("return document.body.scrollHeight")
    scroll_step = (total_height - offset) / scroll_count
    for i in range(1, scroll_count + 1):
        driver.execute_script(f"window.scrollTo(0, {i * scroll_step});")
        time.sleep(pause)
    time.sleep(1)  # Đợi sau khi cuộn xong

# Hàm nhấn nút "Xem thêm" nếu còn
def click_view_more(driver):
    """Tìm và nhấn vào nút 'Xem Thêm' nếu có."""
    try:
        while True:
            view_more_button = driver.find_element(By.CSS_SELECTOR, "button.ViewMoreBtn__Wrapper-sc-qs9ydg-0")
            if view_more_button.is_displayed():
                driver.execute_script("arguments[0].click();", view_more_button)
                time.sleep(1.5)  # Đợi trang tải sau mỗi lần click
            else:
                break
    except (NoSuchElementException, ElementClickInterceptedException):
        return

# Đường dẫn file chứa danh sách cửa hàng
input_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_seller_links.xlsx"

# Đọc danh sách seller links
df = pd.read_excel(input_file)
seller_links = df["Seller Links"].dropna().tolist()[1350:1500]

# Khởi tạo trình duyệt
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)  # Chờ tối đa 10 giây

# Danh sách lưu thông tin
data = []

# Duyệt từng cửa hàng
for link in seller_links:
    try:
        print(f"Đang truy cập: {link}")
        try:
            driver.get(link)
            time.sleep(3)  # Chờ trang tải
        except WebDriverException as e:
            print(f"Lỗi khi mở trang {link}: {e}")
            continue

        # Bước 1: Lấy tên cửa hàng
        try:
            store_name_element = driver.find_element(By.CSS_SELECTOR, "h1.Header__SellerName-sc-vgcg69-2.kgqRrB")
            store_name = store_name_element.text.strip()
            print(f"Tên cửa hàng: {store_name}")
        except NoSuchElementException:
            store_name = None
            print("Không tìm thấy tên cửa hàng.")

        # Bước 2: Click vào tab "Hồ Sơ Cửa Hàng"
        try:
            profile_tab = driver.find_element(By.XPATH, "//div[contains(text(), 'Hồ Sơ Cửa Hàng')]")
            profile_tab.click()
            time.sleep(1.5)  # Chờ trang tải lại sau khi click
            print("Đã click vào tab 'Hồ Sơ Cửa Hàng'")
        except NoSuchElementException:
            print("Không tìm thấy tab 'Hồ Sơ Cửa Hàng', bỏ qua trang này.")
            continue

        # Bước 3: Lấy thông tin cửa hàng
        try:
            info_values = driver.find_elements(By.CSS_SELECTOR, 
                "span.StoreInfo__InfoValue-sc-1un24du-5.GxEKI, "
                "span.StoreInfo__InfoLabel-sc-1un24du-4.jpeWpY")
            extracted_values = [value.text.strip() for value in info_values if value.text.strip()]
            year = extracted_values[2]
            total_products = extracted_values[5] 
            rating = extracted_values[11][:3]
            review_count = extracted_values[12]
            followers = extracted_values[15] 
            chat_response = extracted_values[18]
            print(year, total_products, rating, review_count, followers, chat_response)
        except Exception as e:
            print(f"Lỗi khi lấy thông tin cửa hàng {link}: {e}")
            year, total_products, rating, review_count, followers, chat_response = (None, None, None, None, None, None)

        # Bước 4: Click vào tab "Tất Cả Sản Phẩm"
        try:
            all_products_tab = driver.find_element(By.XPATH, "//div[contains(text(), 'Tất Cả Sản Phẩm')]")
            all_products_tab.click()
            time.sleep(2)  # Chờ trang tải lại sau khi click
        except NoSuchElementException:
            print("Không tìm thấy tab 'Tất Cả Sản Phẩm', bỏ qua sản phẩm.")
            continue

        # Sau khi click vào "Tất Cả Sản Phẩm", kiểm tra và click nút "Xem Thêm" trong danh mục sản phẩm nếu có
        toggler_elements = driver.find_elements(By.CSS_SELECTOR, "div.block a.toggler")
        if toggler_elements:
            toggler_elements[0].click()
            time.sleep(2)  # Đợi nội dung danh mục được mở rộng
        else:
            print("Không có nút 'Xem Thêm' trong danh mục sản phẩm.")

        # Lấy danh mục sản phẩm từ sidebar sau khi đã mở rộng (nếu có)
        category_elements = driver.find_elements(
            By.CSS_SELECTOR,
            "div.block div.list.collapsed div.item-row a, "
            "div.block div.list.opened div.item-row a"
        )
        product_categories = [elem.text.strip() for elem in category_elements] if category_elements else None
        print("Danh mục sản phẩm:", product_categories)

        # Bước 5: Cuộn trang và nhấn "Xem thêm" (cho đến khi hết sản phẩm)
        scroll_page(driver)
        click_view_more(driver)

        # Bước 6: Lấy thông tin tất cả sản phẩm bằng BeautifulSoup
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        product_nodes = soup.select("a.Product__Wrapper-sc-n99tp2-2.Product___StyledWrapper-sc-n99tp2-3.fNzCgC")
        for node in product_nodes:
            name_node = node.select_one("div.Product__Name-sc-n99tp2-7.cfPcgv > h3")
            product_name = name_node.get_text(strip=True) if name_node else None

            product_link = node.get("href")

            rating_node = node.select_one("div.Product__TextRating-sc-n99tp2-25.dDxtAi")
            product_rating = rating_node.get_text(strip=True) if rating_node else None

            sold_node = node.select_one("span.percent")
            product_quantity_sold = sold_node.get_text(strip=True).replace("Đã bán ", "") if sold_node else None

            price_node = node.select_one("div.Product__PriceDiscountValue-sc-n99tp2-9.eWvHOp span")
            product_price = price_node.get_text(strip=True) if price_node else None

            data.append({
                "Seller Link": link,
                "Store Name": store_name,
                "Product Categories": product_categories,
                "Year Joined": year,
                "Total Products": total_products,
                "Rating": rating,
                "Review Count": review_count,
                "Followers": followers,
                "Chat Response Rate": chat_response,
                "Product Name": product_name,
                "Product Link": product_link,
                "Product Rating": product_rating,
                "Price (VND)": product_price,
                "Sold Quantity": product_quantity_sold
            })

        print(data)
    except Exception as e:
        print(f"Lỗi khi xử lý {link}: {e}")

# Đóng trình duyệt
driver.quit()

# Lưu vào Excel
output_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_products_13_1.xlsx"
output_df = pd.DataFrame(data)
output_df = output_df.replace({r'[\x00-\x08\x0B-\x0C\x0E-\x1F]':''}, regex=True)
output_df.to_excel(output_file, index=False)

print(f"\nĐã lưu thông tin vào file: {output_file}")


Tên cửa hàng: Yimitashop
Đã click vào tab 'Hồ Sơ Cửa Hàng'
2021 3+ 0.0 (0 đánh giá) 0 Chưa có
Không có nút 'Xem Thêm' trong danh mục sản phẩm.
Danh mục sản phẩm: ['Túi thời trang nữ', 'Túi thời trang nam', 'Phụ kiện thời trang']


Get rating of each product in store

In [29]:
# Thiết lập để hiển thị tất cả các cột
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)  # Đảm bảo chiều rộng hiển thị không bị giới hạn
pd.set_option('display.max_colwidth', None)  # Hiển thị đầy đủ nội dung trong mỗi cột

In [ ]:
# df = pd.read_excel(r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_products.xlsx")
# df["Product Link"] = "https://tiki.vn" + df["Product Link"]

In [ ]:
# df.to_excel(r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_products_main.xlsx", index=False)

In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, WebDriverException

file_path = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_products_main.xlsx"
df = pd.read_excel(file_path)

In [ ]:
df_copy = df.iloc[:5000]

In [ ]:
# Kiểm tra xem cột "Product Link" có tồn tại không
if "Product Link" not in df_copy.columns:
    raise ValueError("Cột 'Product Link' không tồn tại trong file Excel.")

# Chọn phạm vi sản phẩm cần lấy dữ liệu
product_links = df_copy["Product Link"].dropna().tolist()

# Khởi tạo trình duyệt Selenium
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)

# Tạo các cột mới nếu chưa có
new_columns = ["Overall Rating", "Total Reviews", "5 Stars", "4 Stars", "3 Stars", "2 Stars", "1 Star"]
for col in new_columns:
    if col not in df_copy.columns:
        df_copy[col] = None

# Duyệt qua từng sản phẩm trong phạm vi đã chọn
for link in product_links:
    try:
        # Tìm vị trí hàng trong DataFrame dựa trên "Product Link"
        row_index = df_copy[df_copy["Product Link"] == link].index[0]

        driver.get(link)
        time.sleep(2)  # Chờ trang tải

        # Cuộn xuống trang để tải nội dung (có thể là lazy-load)
        total_height = driver.execute_script("return document.body.scrollHeight")
        for i in range(1, 2):
            driver.execute_script(f"window.scrollTo(0, {i * (total_height / 2.6)});")
            time.sleep(1)  # Chờ trang tải nội dung

        # Sau khi cuộn, kiểm tra nếu trang chứa thông báo "Chưa có đánh giá nào cho sản phẩm này"
        if "Chưa có đánh giá nào cho sản phẩm này" or "Sản phẩm sắp có hàng" or "Sản phẩm đã hết hàng" in driver.page_source:
            print(f"❌ Sản phẩm không có đánh giá: {link}")
            continue

        # Lấy tổng điểm đánh giá
        try:
            overall_rating = driver.find_element(By.CSS_SELECTOR, "div.review-rating__point").text.strip()
        except NoSuchElementException:
            overall_rating = None

        # Lấy tổng số đánh giá
        try:
            total_reviews = driver.find_element(By.CSS_SELECTOR, "div.review-rating__total").text.strip()
            total_reviews = total_reviews.replace("(", "").replace(" đánh giá)", "")
        except NoSuchElementException:
            total_reviews = None

        # Lấy số lượng đánh giá theo sao
        ratings_count = {}
        for i in range(5, 0, -1):
            try:
                rating_xpath = f"//div[@class='review-rating__level'][{6 - i}]/div[@class='review-rating__number']"
                ratings_count[f"{i}_stars"] = driver.find_element(By.XPATH, rating_xpath).text.strip()
            except NoSuchElementException:
                ratings_count[f"{i}_stars"] = None

        # Cập nhật vào DataFrame tại vị trí tìm thấy
        df_copy.at[row_index, "Overall Rating"] = overall_rating
        df_copy.at[row_index, "Total Reviews"] = total_reviews
        df_copy.at[row_index, "5 Stars"] = ratings_count["5_stars"]
        df_copy.at[row_index, "4 Stars"] = ratings_count["4_stars"]
        df_copy.at[row_index, "3 Stars"] = ratings_count["3_stars"]
        df_copy.at[row_index, "2 Stars"] = ratings_count["2_stars"]
        df_copy.at[row_index, "1 Star"] = ratings_count["1_stars"]

        print(f"✅ Đã cập nhật dữ liệu cho sản phẩm: {link}")

    except WebDriverException as e:
        print(f"❌ Lỗi khi truy cập {link}: {e}")
        continue

# Đóng trình duyệt
driver.quit()

# Ghi dữ liệu vào file Excel, không ghi đè dữ liệu cũ
df_copy.to_excel("D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_products_main_1.xlsx", index=False)

print(f"\n📌 Dữ liệu mới đã được thêm vào file: store_products_main_1.xlsx")


C:\Users\CUONG DAT PHAT\AppData\Local\Temp\ipykernel_33016\2698283475.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy[col] = None
C:\Users\CUONG DAT PHAT\AppData\Local\Temp\ipykernel_33016\2698283475.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy[col] = None
C:\Users\CUONG DAT PHAT\AppData\Local\Temp\ipykernel_33016\2698283475.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



✅ Đã cập nhật dữ liệu cho sản phẩm: https://tiki.vn/cap-sac-baseus-cafule-sac-nhanh-dung-cho-iphone-hang-chinh-hang-p142439870.html?spid=142439872
✅ Đã cập nhật dữ liệu cho sản phẩm: https://tiki.vn/tai-nghe-bluetooth-true-wireless-amoi-f9-5-0-cam-ung-van-tay-nang-cap-dock-sac-co-led-bao-pin-kep-hang-nhap-khau-p191684818.html?spid=210637817
✅ Đã cập nhật dữ liệu cho sản phẩm: https://tiki.vn/tai-nghe-nhet-tai-langsdom-super-bass-jm26-hang-nhap-khau-p895314.html?spid=899508
✅ Đã cập nhật dữ liệu cho sản phẩm: https://tiki.vn/tai-nghe-chuyen-game-co-mic-plextone-g25-hang-chinh-hang-p10784603.html?spid=10784607
✅ Đã cập nhật dữ liệu cho sản phẩm: https://tiki.vn/day-cap-sac-nhanh-100w-chuan-pd-2-0-type-c-to-type-c-qc-3-0-hieu-baseus-cafule-type-c-to-type-c-cho-ipad-pro-2018-2020-samsung-huawei-xiaomi-tablet-macbook-laptop-type-c-hang-nhap-khau-p69238917.html?spid=273767858
✅ Đã cập nhật dữ liệu cho sản phẩm: https://tiki.vn/cap-sac-nhanh-truyen-du-lieu-toc-do-cao-baseus-cafule-type-c-cho-